## Check Installation & Import Modules

In [ ]:
from osgeo import gdal
print("Using gdal version", gdal.__version__)

In [ ]:
import pywapor
print("Using pywapor version:", pywapor.__version__)

In [ ]:
%matplotlib inline
import xarray as xr
print("Using xarray version:", xr.__version__)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rioxarray

In [ ]:
from functools import partial
import pywapor.se_root as se_root

In [ ]:
import glob
import os

## Default ETLook Input

### Basic Configuration **(Needs NASA Earthdata Login Details)**

In [ ]:
pywapor.collect.accounts.setup("NASA")

In [ ]:
# User inputs
# Specify data input and output folders

# The default ETLook project output folder
default_project_folder = r"/Users/micky/PycharmProjects/hackathon_pywapor/data_default_pywapor_1"

# The folder containing daily maximum temperature
temp_max_folder = r"/Users/micky/PycharmProjects/hackathon_pywapor/data_custom/temp_max"

# The folder for all netCDF created outputs
# TODO: Check if we need to store xr.Dataset as a.nc file
data_custom_netcdf_folder = r"/Users/micky/PycharmProjects/hackathon_pywapor/data_custom_netcdf"

# The custom side loading method 1 output folder
sl_1_project_folder = r"/Users/micky/PycharmProjects/hackathon_pywapor/data_custom_SL_1"

# Time period: default composite length of pyWAPOR 10 days
timelim = ["2023-01-01", "2023-01-11"]
# Note: composites are also referred to as time_bins
composite_length = "DEKAD"

# Default data source
level = "level_1"
sources = pywapor.general.levels.pre_et_look_levels(level)

# Bounding box of AOI
# TODO: this must be updated
latlim = [-34.1, -33.3] # first value refers to the southern border
lonlim = [18.7, 19.4] # first value refers to the western border

# Input Data Coordinate Reference System
project_crs = "EPSG:4326"

In [ ]:
# Download and prepare input data
ds  = pywapor.pre_et_look.main(default_project_folder, latlim, lonlim, timelim, bin_length=composite_length)

In [ ]:
# To see the file the dataset is stored in
fh = ds.encoding["source"]
print(fh)

In [ ]:
fh = os.path.join(default_project_folder, "et_look_in.nc")

In [ ]:
# To import the file the dataset is stored in (et_look_in.nc) (if you don't want to run pre_et_look again)
# The decode_coords keyword is used to make sure CRS info is loaded correctly.
ds = xr.open_dataset(fh, decode_coords = "all")

In [ ]:
# Take a closer look at the contents of the datasets
# This variable contains a xarray.Dataset which is a Python-package that let's you work with large multi-dimensional datasets.
ds

In [ ]:
# Access the coordinate reference system and boundaries
print("DS CRS: ",ds.rio.crs)
print("DS Bounds: ",ds.rio.bounds())
print("DS Resolution: ",ds.rio.resolution())

In [ ]:
# XArray datasets are also easy to plot and smart enough to automatically fill in the units and the description of the variable.
ds.z.plot()

In [ ]:
# Make a map of a 3-dimensional variable, we will have to select for which time.
ds.t_air_max_24.isel(time_bins = 0).plot()


## Running Default ETLook

In [ ]:
ds_out = pywapor.et_look.main(ds)

In [ ]:
# Check the contents of the new dataset
ds_out

In [ ]:
# Plot the daily evapotranspiration in mm
ds_out.et_24_mm.isel(time_bins = 0).plot()

In [ ]:
# Note these calculations loads the array into working memory
et_data = ds_out.et_24_mm.isel(time_bins = 0).values
print("resolution:", et_data.shape)
print("total pixels:", et_data.size)
print("number of pixels with missing data:", np.sum(np.isnan(et_data)))
print("maximum value: {0:.2f}".format(np.nanmax(et_data)))
print("minimum value: {0:.2f}".format(np.nanmin(et_data)))
print("mean: {0:.2f}".format(np.nanmean(et_data)))
print("median: {0:.2f}".format(np.nanmedian(et_data)))

## TEST SIDELOADING METHOD 1
Instead of passing a string to the sources input parameters of `pywapor.pre_et_look.main` we can also pass a dictionary that specifies which products we would like to use.

### Loading multiple GeoTIFF files into a single xarray.Dataset

Help from: https://docs.dea.ga.gov.au/notebooks/How_to_guides/Opening_GeoTIFFs_NetCDFs.html

Citation:
Krause, C., Dunn, B., Bishop-Taylor, R., Adams, C., Burton, C., Alger, M., Chua, S., Phillips, C., Newey, V., Kouzoubov,
K., Leith, A., Ayers, D., Hicks, A., DEA Notebooks contributors 2021. Digital Earth Australia notebooks and tools 
repository. Geoscience Australia, Canberra. https://doi.org/10.26186/145234

In [ ]:
temp_max_geotiff_list = glob.glob(os.path.join(temp_max_folder, "*.tif"))
print('# of files', len(temp_max_geotiff_list))
temp_max_geotiff_list

In [ ]:
string_slice=(0,10) # extract characters from position 0 to 9
date_strings = [os.path.basename(i)[slice(*string_slice)] for i in temp_max_geotiff_list]
datetime = pd.to_datetime(date_strings)

# Create variable used for time axis
time_var = xr.Variable('time', datetime)

# Load in and concatenate all individual GeoTIFFs
temp_max_geotiffs_da = xr.concat([rioxarray.open_rasterio(i) for i in temp_max_geotiff_list], dim=time_var)

# Convert our xarray.DataArray into a xarray.Dataset
temp_max_geotiffs_ds = temp_max_geotiffs_da.to_dataset('band')

# Rename the variable to a more useful name
temp_max_geotiffs_ds = temp_max_geotiffs_ds.rename({1: 't_air_max'})

temp_max_geotiffs_ds

In [ ]:
# Sort the time variable and select the required date range
temp_max_geotiffs_ds_sorted = temp_max_geotiffs_ds.sortby('time')
temp_max_geotiffs_ds_sorted_timelim = temp_max_geotiffs_ds_sorted.sel(time=slice(timelim[0], timelim[1]))

In [ ]:
temp_max_geotiffs_ds_sorted_timelim

In [ ]:
# Clip to the bounding box
temp_max_geotiffs_ds_sorted_timelim_clip = temp_max_geotiffs_ds_sorted_timelim.rio.clip_box(
    minx= lonlim[0],
    miny= latlim[0],
    maxx= lonlim[1],
    maxy= latlim[1],
    crs=project_crs,
)

# Mask out NoData values
# TODO: Should these rather be filled and not masked? 
temp_max_geotiffs_ds_sorted_timelim_clip_masked = temp_max_geotiffs_ds_sorted_timelim_clip.where(temp_max_geotiffs_ds_sorted_timelim_clip['t_air_max'] != -9999.)

# Reproject to match default pre_et_look output
temp_max_geotiffs_ds_sorted_timelim_clip_masked_project = temp_max_geotiffs_ds_sorted_timelim_clip_masked.rio.reproject_match(ds)

In [ ]:
temp_max_geotiffs_ds_sorted_timelim_clip_masked_project

In [ ]:
temp_max_geotiffs_ds_sorted_timelim_clip_masked_project.t_air_max.isel(time = 0).plot()

In [ ]:
t_air_max_file = os.path.join(data_custom_netcdf_folder, "t_air_max.nc")
# TODO: check if it is okay to use this function (not on the pywapor doc api)
t_air_max_netcdf = pywapor.general.processing_functions.save_ds(
    temp_max_geotiffs_ds_sorted_timelim_clip_masked_project, 
    t_air_max_file, 
    encoding="initiate", 
    label="Testing: t_air_max")

### SIDELOADING METHOD 1

In [ ]:
# Start by loading a defualt configuration for pre_et_look and pre_se_root
et_look_config = pywapor.general.levels.pre_et_look_levels(level = "level_1")
se_root_config = pywapor.general.levels.pre_se_root_levels(level = "level_1")

In [ ]:
# Create a list of variables
meteo_vars = ['t_air_max']

In [ ]:
# Check the default product source and functions
et_look_config['t_air_max']

In [ ]:
# The default enhancers for a specific product can be accessed like this
# TODO: identify if custom products need to keep the enhancers
pywapor.collect.product.GEOS5.default_post_processors('inst3_2d_asm_Nx', 't_air_max')

* For each variable we need to adjust the "products" part.
* Overwrite the list under the "products" key.
* Note: the value of "source" needs to be a function that returns a xr.Dataset which contains a variable called 't_air_max'
* For the product name we can choose a new name!

Older version doc of sideloading: https://www.fao.org/aquastat/py-wapor/notebooks/4_sideloading.html#Sideloading (before se_root model???)

The sideloading function might need to meet certain criteria.
A template function looks like this:

In [ ]:
# TODO: Change custom sideload with more specific parameters
def my_custom_source(folder, latlim, lonlim, timelim, product_name, req_vars, post_processors):
    ...
    # within thie function you can add whatever process you want to execute
    # it must return an xr.Dataset that contains at least the variable for which it is specified in source
    custom_ds = xr.Dataset()
    return ds

In [ ]:
# Step 1: Define a function that can return the dataset
# Note the **kwargs is added to discard the other arguments that are not used in this (very simple) function.
def meteo_sideload(**kwargs):
    meteo_netcdf_file = os.path.join(data_custom_netcdf_folder,"t_air_max.nc")
    meteo_ds = xr.open_dataset(meteo_netcdf_file)
    return meteo_ds

In [ ]:
# Step 2: Put it inside the configuration for each variable.
meteo_config = [{"source": meteo_sideload, "product_name": "METEO_PROVIDED", "enhancers": []}]
for var in meteo_vars:
    et_look_config[var]["products"] = meteo_config
    se_root_config[var]["products"] = meteo_config

In [ ]:
# Now the configuration contains our new product
et_look_config['t_air_max']

Right now the configuration for `"se_root"` inside `et_look_config` still contains the original `level_1` configuration

In [ ]:
et_look_config["se_root"]

In [ ]:
# We can adjust it like this: pass the adjusted se_root_config to et_look_config
se_root_dler = partial(se_root.se_root, sources = se_root_config)
et_look_config["se_root"]["products"][0]["source"] = se_root_dler

In [ ]:
# Now the configuration for "se_root" has been updated too
et_look_config["se_root"]

#### Custom ETLook Input

In [ ]:
# Finally we can start pre_et_look as usual
custom_input_ds = pywapor.pre_et_look.main(sl_1_project_folder, latlim, lonlim, timelim, bin_length=composite_length, sources = et_look_config)

In [ ]:
custom_input_ds

In [ ]:
# Access the coordinate reference system and boundaries
print("Custom CRS: ",custom_input_ds.rio.crs)
print("Custom Bounds: ",custom_input_ds.rio.bounds())
print("Custom Resolution: ",custom_input_ds.rio.resolution())

In [ ]:
custom_input_ds.t_air_max_24.isel(time_bins=0).plot()

#### Running Custom ETLook

In [ ]:
# Run et_look
custom_output_ds = pywapor.et_look.main(custom_input_ds)

In [ ]:
custom_output_ds

In [ ]:
# Plot the daily evapotranspiration in mm
custom_output_ds.et_24_mm.isel(time_bins = 0).plot()

In [ ]:
# Note these calculations loads the array into working memory
custom_et_data = custom_output_ds.et_24_mm.isel(time_bins = 0).values
print("resolution:", custom_et_data.shape)
print("total pixels:", custom_et_data.size)
print("number of pixels with missing data:", np.sum(np.isnan(custom_et_data)))
print("maximum value: {0:.2f}".format(np.nanmax(custom_et_data)))
print("minimum value: {0:.2f}".format(np.nanmin(custom_et_data)))
print("mean: {0:.2f}".format(np.nanmean(custom_et_data)))
print("median: {0:.2f}".format(np.nanmedian(custom_et_data)))